solution to: https://fivethirtyeight.com/features/what-if-robots-cut-your-pizza/
        
with some additional work to calculate variance.  The code in this notebook is from mid 2016, and hence was done in python 2.x.

In [12]:
from __future__ import division
# this imports the do_plots() function into the workbook, albeit in a Jupyter / Ipython way
import numpy as np
from scipy.misc import comb
np.set_printoptions(precision = 3, linewidth=150)


In [15]:
def analytic_variance(n_cuts):
    assert type(n_cuts) is int 
    assert n_cuts > 1
    m = comb(n_cuts, 2)
    if n_cuts < 4:
        unique_pairs_double_pairs = 0
    else:
        unique_pairs_double_pairs = comb(n_cuts, 2) * comb(n_cuts - 2, 2) 
        # this is a multinomial 
    num_pure_duplicates = m
    total_spots = m**2
    # use the outerproduct to see this m x m matrix
    num_mixed_intersections = total_spots - num_pure_duplicates - unique_pairs_double_pairs
    probs = np.array([1/3, 2/15, 1/9])    
    payoffs = np.array([num_pure_duplicates, num_mixed_intersections, unique_pairs_double_pairs]) 
    # not really a payoff so much as a count...
    
    E_of_X_squared = np.dot(probs, payoffs)
    E_of_X = probs[0] * payoffs[0]
    return E_of_X_squared - E_of_X**2

print analytic_variance(10)


26.0


# HOW I DERIVED EXPECTED VALUE 

!['my original sketch '](solution_robo_pizza2_.jpg)


# HOW I DERIVED VARIANCE

Here is how I got that variance calc....

inspiration: page 12 of lectureslides_L07-annotated-slides.pdf (from MITx Probability) -- how to use indicator RVs to get variance of the hat problem

let $m = \binom{n}{2}$

Also it is worth recalling that the expected value of the number of pieces $E[X] = 1 + n + \frac{m}{3}$.  Of that the 1 and the n are fixed for any given n. So for any given n, they shift the mean, but do not contribute to the variation around the mean.  The last term $E[Intersections] = \frac{m}{3}$.  Thus to calculate the variance for our distribution, the only thing that we need to think about it expected intersections.  That is the variance(Pizza Pieces) = variance(intersections of chords).  Thus for the rest of this writeup, we will focus solely on the random variable X, which is the number of chord intersections for some given n_cuts.

- - - -
So we know that $E[X] = \frac{m}{3}$ and in turn we know $(E[X])^2 = \frac{m^2}{9}$ 

To calculate variance we use $var(X) = E[X^2] - (E[X])^2$.  So we need to find a way to calculate $E[X^2]$.

And we can.  It is easiest to visualize what is going on by using my matrix approach do an outer product to create an m x m matrix of indicator random variables (ala the hat problem variance calc).

e.g. for a case where n = 4, and you have chords labelled a, b, c, d

you have a vector, v, of indicator r.v.'s for all possible intersections

in this example

$$ v = \begin{bmatrix}
(a,b)\\ 
(a,c)\\ 
(a,d)\\ 
(b,c)\\ 
(b,d)\\ 
(c,d)\\
\end{bmatrix}$$

so in that top spot, the event of chords a and b intersecting is shown.  In the next spot chords a and c are shown to intersect, and so on.  


Let $h$ be the ones vector with m entries in it

Thus for our example


$$E[X^2] = h^{T}(vv^{T})h = \begin{bmatrix}
1 &  1&  1&  1&  1& 1\\ 
\end{bmatrix}\begin{bmatrix}
(a,b)\cap(a,b)) &  (a,c)\cap(a,b)&  (a,d)\cap(a,b)&  (b,c)\cap(a,b)&  (b,d)\cap(a,b)& (c,d)\cap(a,b)\\ 
(a,b)\cap(a,c) &  (a,c)\cap(a,c)&  (a,d)\cap(a,c)&  (b,c)\cap(a,c)&  (b,d)\cap(a,c)& (c,d)\cap(a,c)\\ 
(a,b)\cap(a,d) &  (a,c)\cap(a,d)&  (a,d)\cap(a,d)&  (b,c)\cap(a,d)&  (b,d)\cap(a,d)& (c,d)\cap(a,d)\\ 
(a,b)\cap(b,c) &  (a,c)\cap(b,c)&  (a,d)\cap(b,c)&  (b,c)\cap(b,c)&  (b,d)\cap(b,c)& (c,d)\cap(b,c)\\ 
(a,b)\cap(b,d)&   (a,c)\cap(b,d)&  (a,d)\cap(b,d)&  (b,c)\cap(b,d)&  (b,d)\cap(b,d)& (c,d)\cap(b,d)\\ 
(a,b)\cap(c,d) &  (a,c)\cap(c,d)&  (a,d)\cap(c,d)&  (b,c)\cap(c,d)&  (b,d)\cap(c,d)& (c,d)\cap(c,d)
\end{bmatrix} \begin{bmatrix}
1 \\  1\\  1\\  1\\  1\\ 1\\ 
\end{bmatrix}$$

*technical note: above and beyond pure linear algebra notation, the above defines* $vv^{T}$  *as the joint occuring of these two events (i.e. of two specific chords intersecting, denoted by a tuple and of two other specific chords intersecting, denoted by a second tuple).  The interest is in the probability of both of these events (or states) occuring at the same time and hence the *$\cap$ * sign is used.  In the interest of concisesness the p() denoting probability of those events occuring has been omitted.  Also note that if the notation $\cap$ (read 'intersect') refering to the set logic intersection of sample space is confusing (especially since we are also interested in another type of intersection -- intersections of chords in a circle), it can instead just be read as AND in the boolean sense.  So for example: $P((a,b) \cap (a,c))$ *can be read as* 
$P((a,b) AND (a,c))$


Looking through the above matrix, there are three key examples.  


(1) $(vv^{T})[0,0] = P((a,b) \cap (a,b)) = P((a,b)) * P((a,b) \mid(a,b)=1) = P(a,b)$  This is duplicative ala diagonal of hat problem so really probability of just one intersection of the chords (a,b).  


(2) $(vv^{T})[1,0] = P((a,b) \cap (a,c)) = P((a,b)) * P((a,c)\mid(a,b)=1)$
*note that in this example 2, they are not independent!*

(3) $(vv^{T})[5,0] = P((a,b) \cap (c,d)) = P((a,b)) * P((c,d)\mid(a,b)=1) = P((a,b)) * P((c,d))$ *in this case they are independent*



These are the three different types of indicator r.v.'s in $vv^{T}$.

Let's store their frequency (or count) of occurrence in one vector with three entries, and the associated indicator r.v. probabilities in another vector with three entries.  Let that first vector be $u$ and the second vector with probabilities be $q$.  

Thus we can rewrite things as $E[X^2] = h^{T}(vv^{T})h = u^{T}q$


**For the initial (i.e. index = 0) entries in u and q:**
All of these from example (1) go along the diagonal of $(vv^{T})$, and since $(vv^{T})$ is an $m$ x $m$ matrix, that means there are m of these entries. Another way of thinking about this is you have n items to selects a pair from.  So you have $\binom{n}{2}$ in that first pair.  Then you select 2 from that first pair and 0 from the residual 'pile'. Thus the calculation is $\binom{n}{2}*\binom{2}{2}*\binom{n-2}{0} = \binom{n}{2} = m$

So $u[0,0] = m$.  We know from calculating $E[X]$ that that the probability of a single pair of chords intersectiong is $\frac{1}{3}$.  Thus $q[0,0] = \frac{1}{3}$

**For the last (i.e. index = 2) entries in u and q:**
The count for these is a multinomial, that can be written as $\binom{n}{2} \binom{n - 2}{2} = m \binom{n - 2}{2}$.  Note that if $n \leq 3$, then this is zero.  The way to think about this one is you have n different chords and you want to select two of them, and then from what remains you want to select two more.  The product of this is the number of ways you you can select two pairs of independent chords from n chords.  (Note that if you look at say the top right and bottom left of $vv^{T}$ you'll see (c,d)(a,b) and then (a,b)(c,d), so while order does not matter within a given tuple, we are saying it does matter between tuples -- if it didn't we'd need to divide our count here by 2!.) 

Since the probabilities are independent, we set $q[2, 0] = \frac{1}{3} * \frac{1}{3} = \frac{1}{9} $


**For the middle (i.e. index = 1) entries in u and q**
Since there are $m^2$ entries in the matrix, we know that this one equals that total minus $u[0,0]$ and minus $u[2,0]$.  That is  $u[1,0] = m^2 - m - \binom{n}{2} \binom{n - 2}{2}$.  Another way of calculating this is to say you select a pair from n objects.  This is $\binom{n}{2}$.  Then you select 1 from that first pair and one from the remaining pile of n - 2 items. The calculation is thus $\binom{n}{2}*\binom{2}{1}*\binom{n-2}{1} = 2\binom{n}{2}*(n-2) = 2m(n-2)$

The probability in there is a touch harder to calculate.  If we use Laurent Lessard's picture (shown here: https://fivethirtyeight.com/features/can-you-figure-out-how-to-beat-roger-federer-at-wimbledon/) we can see there may be 4 out of 15 cases where two chords intersect.  But if we are careful we recognized for say $P((a,b)) * P((a,c)\mid(a,b)=1)$, then the bottom picture applies, and one of the three pictures in the second to bottom row applies.  

That is the second to bottom row has (1) $(a,b) \cap (a,c)$, (2) $(a,b) \cap (b,c)$, (3) $(a,c) \cap (b,c)$, and we want exactly one of these.  An alternative approach inspired by the MITx Probability video (on Beta formula -- my favorite-- ) is to say we have five items $\alpha_1, \beta_1, Z, \alpha_2, \beta_2$.  Any given arrangement chosen uniformly at random has probability = $\frac{1}{5!}$.  Now our only real constraint is that Z must stay in that middle slot, and you cannot have $\alpha_1$ and $\alpha_2$ on the same side as Z (otherwise no intersection with Z's chord will occur). (Note technically the same constraint about $\beta_1, \beta_2$ not being on the same side of Z occurs, but we don't need to worry about it because handling the $\alpha$'s properly will implicitly take care of the $\beta$'s.)

Thus we have 4 spots that we can place $\alpha1$, which leaves 2 spots for $\alpha_2$, which leaves 2 spots for $\beta_1$  and one spot for $\beta_2$.  Thus the total acceptable outcomes $= 4*2*2*1 = 16$.  Thus our entry in the probability vector is given by $q[1,0] = \frac{16}{5!} = \frac{4*2*2*1}{5*4*3*2*1} = \frac{2}{15} $

So to recap.  

Recall $m = \binom{n}{2}$


$ u = \begin{bmatrix}
m \\ 
2m(n-2) \binom{n - 2}{2}\\  m \binom{n - 2}{2}
\end{bmatrix}  $ 

$q = \begin{bmatrix}
1/3\\ 
2/15\\ 
1/9\\
\end{bmatrix}$

- - - 

**Special Handling Note: for ** $n \leq 3$

$ u = \begin{bmatrix}
m \\ 
2m(n-2)\\  0
\end{bmatrix}  $ 

edit: technically this ends up not matter in the simplified function at the end


- - - 
Thus our equation is



$E[X^2] = u^{T}q = \frac{m}{3} + \frac{2(2m(n-2))}{15} + \frac{m \binom{n - 2}{2}}{9} $


$var(X) = E[X^2] - E[X]^2 = u^{T}q - \big(\frac{m}{3}\big)^2 =  \frac{m}{3} + \frac{4m(n-2))}{15} + \frac{m \binom{n - 2}{2}}{9} - \frac{m^2}{9}$


this simplifies to $\frac{n^3 + 2n^2 - 3n}{45}$ which can be factored into $\frac{1}{45} n(n-1)(n+3)$
